In [ ]:
import scanpy as sc
import anndata
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import os
import pandas as pd
import scvelo as scv
import cytopath as cytopath
import seaborn as seaborn
import matplotlib as matplotlib
from plotnine import ggplot, aes, geom_path

In [ ]:
os.chdir('/dartfs-hpc/rc/home/f/f003kcf/CARLIN_analysis/06082022_vsv-ova_infection/fig2_for_paper')

In [ ]:
scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")

In [ ]:
scv.logging.print_version()

# The code used to convert seurat object into an anndata object which is compatible with scVelo was obtained from :https://smorabit.github.io/tutorials/8_velocyto/

In [ ]:
# load sparse matrix:
X = io.mmread("counts_1.mtx")

# create anndata object
adata = anndata.AnnData(
    X=X.transpose().tocsr()
)


In [ ]:
# load cell metadata: 'seurat_clusters' changed to clusters to match the pipeline:
cell_meta = pd.read_csv("metadata_1.csv")

# load gene names:
with open("gene_names_1.csv", 'r') as f:
    gene_names = f.read().splitlines()

# set anndata observations and index obs by barcodes, var by gene names
adata.obs = cell_meta
adata.obs.index = adata.obs['barcode']
adata.var.index = gene_names

# load dimensional reduction:
pca = pd.read_csv("harmony_1.csv")
pca.index = adata.obs.index

# set pca and umap
adata.obsm['X_pca'] = pca.to_numpy()
adata.obsm['X_umap'] = np.vstack((adata.obs['UMAP_1'].to_numpy(), adata.obs['UMAP_2'].to_numpy())).T



In [ ]:
# load loom files for spliced/unspliced matrices for each sample:
##loom files generated using velocyto.py were used here

ldata1 = scv.read('/dartfs-hpc/rc/home/f/f003kcf/CARLIN_analysis/06082022_vsv-ova_infection/LA1_velocity/possorted_genome_bam_XN92C.loom',validate=False )

ldata2 = scv.read('/dartfs-hpc/rc/home/f/f003kcf/CARLIN_analysis/06082022_vsv-ova_infection/LA2_velocity/possorted_genome_bam_J0MP5.loom',validate=False )

ldata3 =scv.read('/dartfs-hpc/rc/home/f/f003kcf/CARLIN_analysis/06082022_vsv-ova_infection/LA3_velocity/possorted_genome_bam_WECGP.loom',validate=False)


#rename barcodes in order to merge:
barcodes = [bc.split(':')[1] for bc in ldata1.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_LA1' for bc in barcodes]
ldata1.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata2.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_LA2' for bc in barcodes]
ldata2.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata3.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_LA3' for bc in barcodes]
ldata3.obs.index = barcodes

# make variable names unique
ldata1.var_names_make_unique()
ldata2.var_names_make_unique()
ldata3.var_names_make_unique()

# concatenate the three loom
ldata = ldata1.concatenate([ldata2, ldata3])




In [ ]:
##Clean obs names to allow for merge. Keep 16nt Cell Id only.
scv.utils.clean_obs_names(adata, id_length=16)
scv.utils.clean_obs_names(ldata,id_length=16)


In [ ]:
# merge matrices into the original adata object
adata = scv.utils.merge(adata, ldata)

adata

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20)
scv.pp.moments(adata, n_pcs=15, n_neighbors=20)


In [ ]:
scv.tl.velocity(adata, mode='stochastic')

#Compute velocity graph based on cosine similarities.
scv.tl.velocity_graph(adata)

#Compute cell-to-cell transition probabilities excluding self transitions 
adata.uns['T_forward'] = scv.utils.get_transition_matrix(adata, self_transitions=False)

In [ ]:
#Compute terminal states (root and end points)
scv.tl.terminal_states(adata)

#Plot velocity grid/stream
scv.pl.velocity_embedding_grid(adata, basis='umap', color='clusters', save='embedding_grid_for_paper.pdf', title='', scale=0.25, dpi=600)
scv.pl.velocity_embedding_stream(adata, color=['root_cells'],density=2, min_mass=2, legend_loc='right margin', size=20, dpi=600, save='inital_states_for_paper')
scv.pl.velocity_embedding_stream(adata, color=['end_points'],density=2, min_mass=2, legend_loc='right margin', size=20, dpi=600, save='terminal_states_for_paper')


In [ ]:
scv.tl.rank_velocity_genes(adata, groupby='clusters', min_corr=.3)

df = scv.DataFrame(adata.uns['rank_velocity_genes']['names'])
df.head()

In [ ]:
##phase portrait for Zeb2
scv.pl.velocity(adata, 'Zeb2', add_outline=False, size=3, dpi=600, save='Zeb2_no_boundary_for_paper.pdf')

In [ ]:
# save dataset as anndata format
adata.write('my_data.h5ad')

In [ ]:
#reopen adata
adata = sc.read_h5ad('my_data.h5ad')

In [ ]:
cytopath.sampling(adata, cluster_key='clusters',end_clusters=['1'],
                  num_cores=os.cpu_count()-1)

In [ ]:
cytopath.trajectories(adata, cluster_freq=0.1,num_cores=os.cpu_count()-1)

In [ ]:
##Cytopath trajectory will be plotted in R to preserve the colors of the seurat clusters
pp=adata.uns['trajectories']['trajectories_coordinates']['1']['trajectory_0_coordinates']

xs = [p[0] for p in pp]
ys= [p[1] for p in pp]
df_1 = pd.DataFrame(ys, xs)
df_1.to_csv('endpoint_1_trajectory_for_paper.csv', index=True)

In [ ]:
seaborn.boxplot(x='clusters', y='end_points', data=adata.obs)

In [ ]:
##for supplement
scv.pl.proportions(adata, save='spliced_unspliced_proprtions_for_paper.pdf')

scv.tl.velocity_confidence(adata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata, c='velocity_confidence', cmap='coolwarm', perc=[5, 95], save='Velocity_confidence_for_paper.pdf',dpi=600)